In [2]:
from scipy.integrate import solve_ivp, odeint
import sys
import time 
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [18, 12]
plt.rcParams['figure.dpi'] = 140
import random
from collections import OrderedDict
from shapely.geometry import Polygon, LineString, Point
np.set_printoptions(linewidth=100)


renata_shape_file_relevant_column = gpd.read_file("./data/china_codes/spatial_match.shp") #shape_file_only_relevant_column/shape_file_only_relevant_column.shp")
tp_df = gpd.GeoDataFrame(renata_shape_file_relevant_column[["GB_Code", "pop_2020", "ADM2_EN", "geometry"]])
tp_df.columns = ['tile_ID', 'population', 'Name', 'geometry']
tp_df['tile_ID'] = tp_df['tile_ID'].fillna(0).astype(int)
tp_df.reset_index(drop=True, inplace=True)
tp_gdf = tp_df.copy() # for getting the geometry
tp_df = pd.DataFrame(tp_df[['tile_ID', 'population', 'Name']])





def filter_data(df):
    mv_ot = df
    mv_ot = mv_ot.fillna(0)
    mv_ot = mv_ot * 50000/100 #44520/100
    mv_ot_id = mv_ot.rename(columns=lambda x: x.split("_")[1])
    column_name_and_index = list(mv_ot_id.columns)
    mv_ot_id.insert(0, "tile_ID", column_name_and_index, True)
    mv_ot_id = mv_ot_id.set_index('tile_ID')
    mv_ot_id= mv_ot_id.to_numpy()
    np.fill_diagonal(mv_ot_id, 0)
    mv_ot_id = pd.DataFrame(mv_ot_id, columns=column_name_and_index)
    mv_ot_id.insert(0, "ID", column_name_and_index, True)
    #mv_ot_id = mv_ot_id.set_index('tile_ID')
    mv_ot_id['ID'] = mv_ot_id['ID'].astype(int)
    #display(mv_ot_id)
    mv_ot_id_filter = mv_ot_id[mv_ot_id['ID'].isin(tp_df['tile_ID'])].copy()
    #display(mv_ot_id_filter)
    mv_ot_id_filter['ID'] = mv_ot_id_filter['ID'].astype(str)
    col_name = mv_ot_id_filter['ID'].tolist()
    mv_ot_id_filter = mv_ot_id_filter.set_index('ID')
    mv_ot_id_filter_ro_col = mv_ot_id_filter[col_name]
    #pd.concat([mv_ot_id_filter['ID'], mv_ot_id_filter[col_name]], axis=1)
    #mv_ot_id_filter_ro_col =  mv_ot_id_filter_ro_col.set_index('ID')
    return mv_ot_id_filter_ro_col

def tp_filter(df, epsilon_0):
    epsilon_0 = epsilon_0.reset_index(drop=False)
    epsilon_0 = epsilon_0[['ID', '110000']]
    epsilon_0 = epsilon_0.set_index('ID')
    df = df.drop('0', axis = 0)
    #display(epsilon_0)
    tp_filter = pd.concat([epsilon_0, df], axis=1)
    #display(tp_filter)
    tp_arr = tp_filter['population'].to_numpy()
    return tp_arr


mv_ot = pd.read_csv('./data/move_out_mean_2021_01_19_2022_01_18_.csv', index_col=0)
epsilon_0 = filter_data(mv_ot)
#display(epsilon_0)
mv_in = pd.read_csv('./data/move_in_mean_2021_01_19_2022_01_18_.csv', index_col=0)
epsilon_1 = filter_data(mv_in)
#epsilon_1
Percentage_list = [100, 95, 75, 50, 30, 25, 20, 15, 10, 5, 1]
#Percentage_list = [15]
inf_time_for_100 = []
for percentage in Percentage_list:
    print (percentage)
    epsilon = (epsilon_0 + epsilon_1)/2.0
    '''interventions at two pivot locations'''
    pr = pd.read_csv('./data/Weighted_Personalized_PageRank_for_Wuhan_outflow.csv', index_col='Name_ID')
    pr = pr.sort_values('WeightedPersonalizedPagerank', ascending=False)
    pr = pr.iloc[:10, : ]
    pr_list = list(pr.index.values)
    pr_list_code = [x.split('_')[1] for x in pr_list]
    
    #print (epsilon[pr_list_code[0]])

    epsilon[pr_list_code[0]] = epsilon[pr_list_code[0]]*(percentage/100)




    tp_for_pop_filter = tp_df.copy()
    tp_for_pop_filter = tp_for_pop_filter.rename(columns={'tile_ID': 'ID'})
    tp_for_pop_filter['ID'] = tp_for_pop_filter['ID'].astype(str)
    tp_for_pop_filter = tp_for_pop_filter.set_index('ID')
    tp = tp_filter(tp_for_pop_filter, epsilon_0) # get population the same order of ID in the flow frame


    # print (tp[0:no_of_loc])
    # tp = tp[0:no_of_loc]
    # print(tp)
    #tp = np.array([18827262.0, 11090783.0, 10163788.0, 7577289.0, 2987605.0])
    epsilon = epsilon/tp
    epsilon= epsilon.to_numpy()
    np.fill_diagonal(epsilon, 0)
    n=len(epsilon)
    pi = (np.ones(n*n).reshape(n,n))*(1/7)

    #tc = 3  # time between contacts in days 
    tl = 5.5  # latent period of the infection
    tr = 10  # recovery time in days
    te = 60*365 # Human life expectancy in days (60 yr) 

#     R0_list = [3]
#     for rnot in R0_list:
    R0 = 3.0

    #beta = 1 / tc  # contact rate in per daymu
    sigma = 1 /tl  # latency rate per day
    gamma = 1 / tr # recovery rate in per day
    mu = 1/te # Natural human death rate (in days^-1)
    beta = R0*(gamma+mu)
    #beta = R0*(gamma) #natural births and deaths are not considered in our model 

    t0 = 0.0 # start time
    tn = 356 # No_of_days
    time_sp = 1 # time increment is one day


    S0=np.zeros(n, dtype='float128')
    E0=np.zeros(n, dtype='float128')
    I0=np.zeros(n, dtype='float128')
    R0=np.zeros(n, dtype='float128')

    # s0=np.random.randint(1,5,(1,n*n)).reshape(n,n) # Sij random num b/w 1 and 5
    # e0=np.random.randint(1,5,(1,n*n)).reshape(n,n) # Eij random num b/w 1 and 5
    # i0=np.random.randint(1,5,(1,n*n)).reshape(n,n) # Iij random num b/w 1 and 5
    # r0=np.random.randint(1,5,(1,n*n)).reshape(n,n) # Rij random num b/w 1 and 5

    s0= np.zeros(n*n).reshape(n,n)
    e0= np.zeros(n*n).reshape(n,n)
    i0= np.zeros(n*n).reshape(n,n)
    r0= np.zeros(n*n).reshape(n,n)

    np.fill_diagonal(s0, 0)
    np.fill_diagonal(e0, 0)
    np.fill_diagonal(i0, 0)
    np.fill_diagonal(r0, 0)


    #for i in range(0, len(tp)):
    #E_temp = np.random.randint(100,150,(1,n)).reshape(-1)
    E_temp = np.zeros(n)
    #I_temp = np.random.randint(1500,2000,(1,n)).reshape(-1)

    time = []
    inf_loc =[]
    m = 157 # location of Wuhan
    #for i in range(m,m+1):
    #x = i
    #print ('Location:', i)
    I_temp = np.zeros(n)
    I_temp[m] = 100
    #print (I_temp)
    #I_temp = np.random.randint(1500,2000,(1,n)).reshape(-1)
    #print (E_temp)

    S0 = tp + s0.sum(axis=1) - e0.sum(axis=1) - i0.sum(axis=1) -r0.sum(axis=1) - E_temp - I_temp
    #S0 = tp 
    E0 =  E_temp 
    I0 = I_temp 
    #R0 = r0.reshape(n,n).sum(axis=1)

    # print ('Sii0:', S0)
    # print ('Eii0:', E0)
    # print ('Iii0:', I0)
    # print ('Rii0:', R0)
    # print ('sumSij0:', s0.sum(axis=1))
    # print ('sumEij0:', e0.sum(axis=1))
    # print ('sumIij0:', i0.sum(axis=1))
    # print ('sumRij0:', r0.sum(axis=1))

    #print('Sii0 + Eii0 + Iii0 + Rii0 + sumSij0 + sumEij0 + sumIij0 + sumRij0:', S0+E0+I0+R0+s0.sum(axis=1)+e0.sum(axis=1)+i0.sum(axis=1)+r0.sum(axis=1))


    sij = s0.reshape(-1)
    eij = e0.reshape(-1)
    iij = i0.reshape(-1)
    rij = r0.reshape(-1)


    ics = np.hstack((S0,E0,I0,R0,sij,eij,iij,rij))

    #print (ics.dtype)



    def seir_meta(t, ics, epsilon, pi):
        Sii = ics[0:n]; Eii = ics[n:2*n]
        Iii = ics[(2*n):3*n]; Rii = ics[(3*n):4*n]
        Sij = ics[(4*n+(0*n**2)):(4*n+(1*n**2))]
        Eij = ics[(4*n+(1*n**2)):(4*n+(2*n**2))]
        Iij = ics[(4*n+(2*n**2)):(4*n+(3*n**2))]
        Rij = ics[(4*n+(3*n**2)):(4*n+(4*n**2))]
        Sij.shape = (n,n); Eij.shape = (n,n); Iij.shape = (n,n); Rij.shape = (n,n)

        axis = 1
        sumIij = Iii + Iij.sum(axis=axis) # sumIij... no: of infected people in each location from all other loc ([len[N]])
        sumNij = Sii + Eii + Iii + Rii + Sij.sum(axis=axis) + Eij.sum(axis=axis) + Iij.sum(axis=axis) + Rij.sum(axis=axis)
        ''' Xij.sum(axis=1) are the arrays of population from all j's to a particular i '''

        Nij = Sij + Eij + Iij + Rij


        dSii_dt = mu*sumNij -beta*Sii*(sumIij/sumNij) - (epsilon*Sii).sum(axis=0) + (pi*Sij).sum(axis=0) - mu*Sii
        dSij_dt = mu*Nij -beta*Sij*(sumIij/sumNij)  + (epsilon*Sii)  - pi*Sij - mu*Sij
        dEii_dt = beta*Sii*(sumIij/sumNij) - (epsilon*Eii).sum(axis=0) + (pi*Eij).sum(axis=0) - sigma * Eii - mu*Eii
        dEij_dt = beta*Sij*(sumIij/sumNij) + (epsilon*Eii) - pi*Eij - sigma*Eij - mu*Eij
        dIii_dt = sigma * Eii  - (epsilon*Iii).sum(axis=0) + (pi*Iij).sum(axis=0) - gamma*Iii - mu*Iii
        dIij_dt = sigma * Eij  + (epsilon*Iii) - pi*Iij - gamma*Iij  - mu*Iij
        dRii_dt = gamma*Iii - (epsilon*Rii).sum(axis=0) + (pi*Rij).sum(axis=0) - mu*Rii
        dRij_dt = gamma*Iij  + (epsilon*Rii) - pi*Rij - mu*Rij

        dSij_dt_r = dSij_dt.reshape(-1) #Flattening (n x n) array -- Converting the (n x n) array into a 1D array
        dEij_dt_r = dEij_dt.reshape(-1) #Flattening (n x n) array -- Converting the (n x n) array into a 1D array
        dIij_dt_r = dIij_dt.reshape(-1) #Flattening (n x n) array -- Converting the (n x n) array into a 1D array
        dRij_dt_r = dRij_dt.reshape(-1) #Flattening (n x n) array -- Converting the (n x n) array into a 1D array

        r = np.hstack((dSii_dt, dEii_dt, dIii_dt, dRii_dt, dSij_dt_r, dEij_dt_r, dIij_dt_r, dRij_dt_r))

        return r

    p = (epsilon, pi)
    t_span = (t0, tn)
    t = np.arange(t0, tn+time_sp, time_sp)

    # tic = time.time()
    # result_odeint = odeint(seir_meta, ics, t, p, tfirst=True)
    # toc = time.time()
    # print("Time Elapsed for ODEINT:", toc - tic)


    #tic = time.time()
    result_solve_ivp = solve_ivp(seir_meta, t_span, ics, args=p,
                                   method='DOP853', dense_output=True, t_eval=t, rtol= 1e-3, atol=1e-6) #DOP853
    # toc = time.time()
    # print("Time Elapsed for solve_ivp: ", toc - tic)

    #State DataFrame Creation
    col_name = list(mv_ot.columns)
    name = [x.split('_')[0] for x in col_name]
    code = [x.split('_')[1] for x in col_name]
    code_filter = list(epsilon_0.columns)
    #print(len(code), len(code_filter))
    index = []
    for i, x in enumerate(code):
        for j in code_filter:
            if x == j:
                index.append(i)

    name_filter = []
    for i, x in enumerate(index):
        name_filter.append(name[x])



    level_1_label = [n+'_'+c for n,c in zip(name_filter,code_filter)]
    level_1_label = level_1_label[0:n]
    level_2_label = ['S_ii','S_ij','E_ii','E_ij','I_ii','I_ij','R_ii','R_ij']
    #level_2_label = ['S_ii','E_ii','I_ii','R_ii']
    S = result_solve_ivp.y[0:n, : ].T; E = result_solve_ivp.y[n:2*n, : ].T
    I = result_solve_ivp.y[2*n:3*n, : ].T; R = result_solve_ivp.y[3*n:4*n, : ].T


    Sij = result_solve_ivp.y[(4*n+(0*n**2)):(4*n+(1*n**2))].T
    Eij = result_solve_ivp.y[(4*n+(1*n**2)):(4*n+(2*n**2))].T
    Iij = result_solve_ivp.y[(4*n+(2*n**2)):(4*n+(3*n**2))].T
    Rij = result_solve_ivp.y[(4*n+(3*n**2)):(4*n+(4*n**2))].T

    Sij = Sij.reshape(Sij.shape[0],n,n); Eij = Eij.reshape(Eij.shape[0],n,n) 
    Iij = Iij.reshape(Iij.shape[0],n,n); Rij = Rij.reshape(Rij.shape[0],n,n)
    #x.shape is a 2-tuple which represents the shape of x,  
    #x.shape[0] gives the first element in that tuple

    sumSij = Sij.sum(axis=1); sumEij = Eij.sum(axis=1)  #X's came to i from all js
    sumIij = Iij.sum(axis=1); sumRij = Rij.sum(axis=1)  #X's came to i from all js

    header = pd.MultiIndex.from_product([level_1_label, level_2_label], names=['Location','State'])
    #fortran_order = np.reshape(np.vstack([S,E,I,R]), (len(S), -1), order = 'F') #reshape the data in Fortran order, before creating the dataframe
    fortran_order = np.reshape(np.vstack([S, sumSij,E,sumEij,I,sumIij,R,sumRij]), (len(S), -1), order = 'F')
    df = pd.DataFrame(fortran_order, columns = header)
    df.index.name = 'Time'

    #tot_te = time.time()
    #print("Total Time Elapsed: ", tot_te - tot_t)

    df1 = df.iloc[:, df.columns.get_level_values(0)=='Beijingshi_110000']
    #df2 = df1.iloc[:, df1.columns.get_level_values(1)=='I_ii']
    df1.columns = df1.columns.get_level_values('State')
    time_temp = df1[df1['I_ii'] + df1['I_ij'] >=100.].index.values
    #print(x)
    if len(time_temp) != 0:
        time.append(time_temp[0])
    else:
        time.append(999)
    inf_time_for_100.append(time_temp[0])


# Create a pandas DataFrame from the two lists
data = {'percentage_num': Percentage_list, 'Wuhan_only_Restriting_flow': inf_time_for_100}
df_100_inf_time_Wuhan_only = pd.DataFrame(data)

df_100_inf_time_Wuhan_only

100
95
75
50
30
25
20
15
10
5
1


,percentage_num,Wuhan_only_Restriting_flow
0,100,59
1,95,60
2,75,62
3,50,65
4,30,69
5,25,71
6,20,73
7,15,76
8,10,80
9,5,86
